In [1]:
import numpy as np
import open3d as o3d
from pyntcloud import PyntCloud

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import pandas as pd

In [13]:
cloud = pd.read_csv('../data/birmingham_blocks/7_test/chunk_0.txt')

In [14]:
o3d.io.read_point_cloud('../data/birmingham_blocks/7_test/chunk_0.txt', format='xyzi')

[Open3D WARNING] Read geometry::PointCloud failed: unknown file extension for ../data/birmingham_blocks/7_test/chunk_0.txt (format: xyzi).


PointCloud with 0 points.

In [16]:
import pandas as pd

def csv_to_ply(df, output_ply):
    """
    Convert a CSV file (with x, y, z, red, green, blue, class columns) to a PLY file.

    Args:
        input_csv (str): Path to the input CSV file.
        output_ply (str): Path to the output PLY file.
    """

    # Ensure the correct columns are present
    required_columns = ['x', 'y', 'z', 'red', 'green', 'blue', 'class']
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"CSV must contain the following columns: {required_columns}")

    # Get vertex count
    vertex_count = len(df)

    # Write PLY header
    header = f"""ply
format ascii 1.0
comment Created by Python Script
element vertex {vertex_count}
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
property float class
end_header
"""

    # Write PLY file
    with open(output_ply, 'w') as ply_file:
        ply_file.write(header)
        for _, row in df.iterrows():
            # Format the row to match the PLY requirements
            ply_file.write(f"{row['x']} {row['y']} {row['z']} "
                           f"{int(row['red'])} {int(row['green'])} {int(row['blue'])} "
                           f"{row['class']}\n")

    print(f"PLY file successfully created: {output_ply}")

In [17]:
csv_to_ply(cloud, output_ply='../data/birmingham_blocks/7_test/test.ply')

PLY file successfully created: ../data/birmingham_blocks/7_test/test.ply


In [18]:
from urb3d.segmentation.dataset import ChunkedPointCloudDataset

In [23]:
dataset = ChunkedPointCloudDataset('../data/birmingham_blocks/7_test')

In [26]:
dataset.__getitem__(0)

1

In [ ]:
import os

merged_cloud = None
predictions = np.array([])

for idx in range(len(os.listdir(input_chunks_dir))):
    chunk_preds = preds[idx][0].cpu().numpy()
    predictions = np.concatenate([predictions, chunk_preds])
    chunk_cloud = PyntCloud.from_file(f"{input_chunks_dir}/chunk_{idx}.csv").points[
        ['x', 'y', 'z']]  # chunk[0][0].cpu().numpy().transpose()
    merged_cloud = chunk_cloud if merged_cloud is None else np.concatenate([merged_cloud, chunk_cloud])

# combine cloud and predictions
new_cloud = pd.DataFrame(np.concatenate([merged_cloud, predictions[..., np.newaxis]], axis=1),
                         columns=['x', 'y', 'z', 'class'])
cloud = PyntCloud(new_cloud)